# Home Credit Prediction: Data Cleaning of Current Application Table


For an up-to-date version / full view of the plotly - plots, please, go to

Data Cleaning - Current Application Train: https://drive.google.com/file/d/17PZtY-xD-6AbF_9B0CkhxnD25WAUhnzP/view?usp=sharinghttps://drive.google.com/file/d/17PZtY-xD-6AbF_9B0CkhxnD25WAUhnzP/view?usp=sharing


List of all notebooks and resources for this project: https://drive.google.com/file/d/1Z8vPNZAcivWOxeh3UKFfeARbQCMkQ_NR/view?usp=sharing

## Import Modules

In [ ]:
%%capture
#! pip install -q kaggle
#! pip install -q pingouin
! pip install -q scikit-optimize

In [ ]:
import numpy as np
import pandas as pd

import sys
import os
import warnings
from importlib import reload

from dask import dataframe as dd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#
##from statsmodels.stats.proportion import proportions_ztest
#import pingouin as pg
#
#
#
from google.colab import drive
drive.mount("/content/gdrive")

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_rows')

Mounted at /content/gdrive


In [ ]:
home_folder = '/content/gdrive/MyDrive/Colab Notebooks/Portfolio/ML_HomeCredit_DefaultRiskEvaluation/'

### Functions

The Python-file with the functions is at
https://drive.google.com/file/d/17IchsTGy2QI9sq0LTIvGvxAk2mrWs4Xz/view?usp=sharing

In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.append(home_folder)
import driskfunc as dfunc

# 1. Load and Update Data

data source: https://storage.googleapis.com/341-home-credit-default/home-credit-default-risk.zip

description: https://storage.googleapis.com/341-home-credit-default/Home%20Credit%20Default%20Risk.pdf

In [ ]:
HCdescr = pd.read_csv(home_folder+'data/HomeCredit_columns_description.csv', encoding='latin1') #, dtype=dtype)


In [ ]:
HCdescr.loc[HCdescr.Table == 'application_{train|test}.csv']

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)",NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,NaN


In [ ]:
csv_app_train = home_folder+'data/application_train.csv'
csv_app_test = home_folder+'data/application_test.csv'

HCapp_train_dd = dd.read_csv(csv_app_train) #, dtype=dtype)
HCapp_test_dd = dd.read_csv(csv_app_test) #, dtype=dtype)

In [ ]:
HCapp_train_dd.npartitions

2

In [ ]:
HCapp_test_dd.npartitions

1

In [ ]:
HCapp_train_dd.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
HCapp_test_dd.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,<NA>,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,<NA>,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>,<NA>,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,<NA>,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>,<NA>,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary spec

In [ ]:
HCapp_train_dd.tail()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
153678,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,0.032561,-9327,-236,-8456.0,-1982,NaN,1,1,0,1,0,0,Sales staff,1.0,1,1,THURSDAY,15,0,0,0,0,0,0,Services,0.145570,0.681632,NaN,0.2021,0.0887,0.9876,0.8300,0.0202,0.22,0.1034,0.6042,0.2708,0.0594,0.1484,0.1965,0.0753,0.1095,0.1008,0.0172,0.9782,0.7125,0.0172,0.0806,0.0345,0.4583,0.0417,0.0094,0.0882,0.0853,0.0,0.0125,0.2040,0.0887,0.9876,0.8323,0.0203,0.22,0.1034,0.6042,0.2708,0.0605,0.1509,0.2001,0.0757,0.1118,reg oper account,block of flats,0.2898,"Stone, brick",No,0.0,0.0,0.0,0.0,-273.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
153679,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.025164,-20775,365243,-4388.0,-4090,NaN,1,0,0,1,1,0,<NA>,1.0,2,2,MONDAY,8,0,0,0,0,0,0,XNA,NaN,0.115992,NaN,0.0247,0.0435,0.9727,0.6260,0.0022,0.00,0.1034,0.0833,0.1250,0.0579,0.0202,0.0257,0.0000,0.0000,0.0252,0.0451,0.9727,0.6406,0.0022,0.0000,0.1034,0.0833,0.1250,0.0592,0.0220,0.0267,0.0,0.0000,0.0250,0.0435,0.9727,0.6310,0.0022,0.00,0.1034,0.0833,0.1250,0.0589,0.0205,0.0261,0.0000,0.0000,reg oper account,block of flats,0.0214,"Stone, brick",No,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
153680,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,Unaccompanied,Working,Higher education,Separated,House / apartment,0.005002,-14966,-7921,-6737.0,-5150,NaN,1,1,0,1,0,1,Managers,1.0,3,3,THURSDAY,9,0,0,0,0,1,1,School,0.744026,0.535722,0.218859,0.1031,0.0862,0.9816,0.7484,0.0123,0.00,0.2069,0.1667,0.2083,NaN,0.0841,0.9279,0.0000,0.0000,0.1050,0.0894,0.9816,0.7583,0.0124,0.0000,0.2069,0.1667,0.

In [ ]:
size_train = [HCapp_train_dd.shape[0].compute(),  HCapp_train_dd.shape[1]]
size_test  = [HCapp_test_dd.shape[0].compute(),  HCapp_test_dd.shape[1]]

print('The "application train" dataset has', size_train[0], 'rows and', size_train[1], 'features.')
print('The "application test" dataset has', size_test[0], 'rows and', size_test[1], 'features.')

The "application train" dataset has 307511 rows and 122 features.
The "application test" dataset has 48744 rows and 121 features.


In [ ]:
#cat_acc_feat = LCacc_dd.select_dtypes(include=np.object).columns.tolist()
#cat_rej_feat = LCrej_dd.select_dtypes(include=np.object).columns.tolist()
#num_acc_feat = LCacc_dd.select_dtypes(include=np.number).columns.tolist()
#num_rej_feat = LCrej_dd.select_dtypes(include=np.number).columns.tolist()
#

Reread data with objects turned into categories to save RAM:

In [ ]:
#dtype_acc = {}
#for obj in cat_acc_feat:
#    dtype_acc[obj] = 'category'
#dtype_acc['policy_code'] = 'category'
#
#dtype_rej = {}
#for obj in cat_rej_feat:
#    dtype_rej[obj] = 'category'
#dtype_acc['Policy Code'] = 'category'

In [ ]:
#LCacc_dd = dd.read_csv(csv_accepted, dtype=dtype_acc)
#LCrej_dd = dd.read_csv(csv_rejected, dtype=dtype_rej)

# 2. Data Cleaning

* Handling missing values.
* Removing duplicate samples and features.
* Remove unneccessary columns/rows.
* Treating (here rather checking) the outliers.

## Check Missing Values and Duplicates

Overview of amounts of Nan and of data type:

In [ ]:
dfunc.count_dtypes(HCapp_train_dd, name = 'HCapp_train_dd')
dfunc.count_dtypes(HCapp_test_dd, name = 'HCapp_test_dd')


The dataset HCapp_train_dd has:
65 features of type float64.
41 features of type int64.
16 features of type string.

The dataset HCapp_test_dd has:
65 features of type float64.
40 features of type int64.
16 features of type string.


In [ ]:
%%time
%reload_ext autoreload

nan_overview_train = dfunc.nan_type_overview_dd(HCapp_train_dd, size_train[0]).round(1)


CPU times: user 6.26 s, sys: 898 ms, total: 7.16 s
Wall time: 4.44 s


In [ ]:
nan_overview_test = dfunc.nan_type_overview_dd(HCapp_test_dd, size_test[0]).round(1)

In [ ]:
merged_nan = nan_overview_train.merge(nan_overview_test, left_index=True,
                                      right_index=True, suffixes=('_train', '_test'),
                                      how='outer')
merged_nan.style.background_gradient(cmap="Blues")

,type_train,NaN[abs]_train,NaN[%]_train,type_test,NaN[abs]_test,NaN[%]_test
AMT_ANNUITY,float64,12,0.000000,float64,24.000000,0.000000
AMT_CREDIT,float64,0,0.000000,float64,0.000000,0.000000
AMT_GOODS_PRICE,float64,278,0.100000,float64,0.000000,0.000000
AMT_INCOME_TOTAL,float64,0,0.000000,float64,0.000000,0.000000
AMT_REQ_CREDIT_BUREAU_DAY,float64,41519,13.500000,float64,6049.000000,12.400000
AMT_REQ_CREDIT_BUREAU_HOUR,float64,41519,13.500000,float64,6049.000000,12.400000
AMT_REQ_CREDIT_BUREAU_MON,float64,41519,13.500000,float64,6049.000000,12.400000
AMT_REQ_CREDIT_BUREAU_QRT,float64,41519,13.500000,float64,6049.000000,12.400000
AMT_REQ_CREDIT_BUREAU_WEEK,float64,41519,13.500000,float64,6049.000000,12.400000
AMT_REQ_CREDIT_BUREAU_YEAR,float64,41519,13.500000,float64,6049.000000,12.400000


### Drop Missing Value Columns

Drop all columns with too many missing values:

In [ ]:
#cut_off_percentage = 70.
#
#too_many_nan_acc = nan_overview_acc['NaN[%]'
#                ].loc[nan_overview_acc['NaN[%]']
#                > cut_off_percentage].round(1)
#
#len(too_many_nan_acc), too_many_nan_acc.index.to_list()

In [ ]:
#LCacc_dd = LCacc_dd.drop(columns=too_many_nan_acc.index.to_list())


In [ ]:
#%time
#%reload_ext autoreload
#
#nan_overview_acc_drop = lfunc.nan_type_overview_dd(LCacc_dd, size_acc[0])
#nan_overview_acc_drop.round(1).style.background_gradient(cmap="Blues")

### Duplicates Check

In [ ]:
%reload_ext autoreload

HCapp_train_dup = dfunc.get_dup_dd(HCapp_train_dd, name='HCapp_train_dd', size=size_train[0])
HCapp_test_dup = dfunc.get_dup_dd(HCapp_test_dd, name='HCapp_test_dd', size=size_test[0])

Total number of duplicates in " HCapp_train_dd " : 0 ( 0.0 %).
Total number of duplicates in " HCapp_test_dd " : 0 ( 0.0 %).


## Drops based on Statistical Overview

### Overview

In [ ]:
HCapp_train_dd.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
SK_ID_CURR,307511.0,278180.5,102790.2,100002.0,144529.8,233788.2,322922.5,456255.0
TARGET,307511.0,0.1,0.3,0.0,0.0,0.0,0.0,1.0
CNT_CHILDREN,307511.0,0.4,0.7,0.0,0.0,0.0,1.0,19.0
AMT_INCOME_TOTAL,307511.0,168797.9,237123.1,25650.0,112500.0,148500.0,202500.0,117000000.0
AMT_CREDIT,307511.0,599026.0,402490.8,45000.0,270000.0,513531.0,808650.0,4050000.0
AMT_ANNUITY,307499.0,27108.6,14493.7,1615.5,16564.5,24925.5,34596.0,258025.5
AMT_GOODS_PRICE,307233.0,538396.2,369446.5,40500.0,238500.0,450000.0,679500.0,4050000.0
REGION_POPULATION_RELATIVE,307511.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
DAYS_BIRTH,307511.0,-16037.0,4364.0,-25229.0,-19670.2,-15727.0,-12388.0,-7489.0
DAYS_EMPLOYED,307511.0,63815.0,141275.8,-17912.0,-2753.0,-1213.0,-289.0,365243.0


In [ ]:
HCapp_train_dd.loc[HCapp_train_dd.DAYS_EMPLOYED>0.0].head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
8,100011,0,Cash loans,F,N,Y,0,112500.000,1019610.0,33826.5,913500.0,Children,Pensioner,Secondary / secondary special,Married,House / apartment,0.018634,-20099,365243,-7427.0,-3514,NaN,1,0,0,1,0,0,<NA>,2.0,2,2,WEDNESDAY,14,0,0,0,0,0,0,XNA,0.587334,0.205747,0.751724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>,<NA>,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
11,100015,0,Cash loans,F,N,Y,0,38419.155,148365.0,10678.5,135000.0,Children,Pensioner,Secondary / secondary special,Married,House / apartment,0.015221,-20417,365243,-5246.0,-2512,NaN,1,0,0,1,1,0,<NA>,2.0,2,2,FRIDAY,7,0,0,0,0,0,0,XNA,0.722044,0.555183,0.652897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>,<NA>,0.0,0.0,0.0,0.0,-2396.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
23,100027,0,Cash loans,F,N,Y,0,83250.000,239850.0,23850.0,225000.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.006296,-24827,365243,-9012.0,-3684,NaN,1,0,0,1,1,0,<NA>,2.0,3,3,FRIDAY,12,0,0,0,0,0,0,XNA,NaN,0.624305,0.669057,0.1443,0.0848,0.9876,0.830,0.1064,0.14,0.1207,0.3750,0.4167,0.2371,0.1173,0.1484,0.0019,0.0007,0.1261,0.0754,0.9876,0.8367,0.0000,0.1208,0.1034,0.3750,0.4167,0.2425,0.1102,0.0923,0.0000,0.0000,0.1457,0.0848,0.9876,0.8323,0.1071,0.14,0.1207,0.3750,0.4167,0.2412,0.1193,0.1510,0.0019,0.0007,org spec account,block of flats,0.1640,Mixed,No,0.0,0.0,0.0,0.0,-795.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
38,100045,0,Cash loans,F,N,Y,0,9

In [ ]:
HCapp_train_dd.loc[HCapp_train_dd.DAYS_EMPLOYED>=0.0].DAYS_EMPLOYED.value_counts().compute()

DAYS_EMPLOYED
365243    55374
0             2
Name: count, dtype: int64

In [ ]:
HCapp_train_dd_obj = HCapp_train_dd.describe(exclude=np.number).compute().T
HCapp_train_dd_obj['freq[%]'] = (HCapp_train_dd_obj['freq'].astype('float')/
                                 HCapp_train_dd_obj['count'].astype('float')*100)
HCapp_train_dd_obj.round(1)

,unique,count,top,freq,freq[%]
NAME_CONTRACT_TYPE,2,307511,Cash loans,278232,90.5
CODE_GENDER,3,307511,F,202448,65.8
FLAG_OWN_CAR,2,307511,N,202924,66.0
FLAG_OWN_REALTY,2,307511,Y,213312,69.4
NAME_TYPE_SUITE,7,306219,Unaccompanied,248526,81.2
NAME_INCOME_TYPE,8,307511,Working,158774,51.6
NAME_EDUCATION_TYPE,5,307511,Secondary / secondary special,218391,71.0
NAME_FAMILY_STATUS,6,307511,Married,196432,63.9
NAME_HOUSING_TYPE,6,307511,House / apartment,272868,88.7
OCCUPATION_TYPE,18,211120,Laborers,55186,26.1


In [ ]:
HCapp_test_dd_obj = HCapp_test_dd.describe(exclude=np.number).compute().T
HCapp_test_dd_obj['freq[%]'] = (HCapp_test_dd_obj['freq'].astype('float')/
                                HCapp_test_dd_obj['count'].astype('float')*100)
HCapp_test_dd_obj.round(1)

,unique,count,top,freq,freq[%]
NAME_CONTRACT_TYPE,2,48744,Cash loans,48305,99.1
CODE_GENDER,2,48744,F,32678,67.0
FLAG_OWN_CAR,2,48744,N,32311,66.3
FLAG_OWN_REALTY,2,48744,Y,33658,69.1
NAME_TYPE_SUITE,7,47833,Unaccompanied,39727,83.1
NAME_INCOME_TYPE,7,48744,Working,24533,50.3
NAME_EDUCATION_TYPE,5,48744,Secondary / secondary special,33988,69.7
NAME_FAMILY_STATUS,5,48744,Married,32283,66.2
NAME_HOUSING_TYPE,6,48744,House / apartment,43645,89.5
OCCUPATION_TYPE,18,33139,Laborers,8655,26.1


In [ ]:
#cut_off_percentage = 95.
#
#extreme_majority_class_acc = HCapp_train_dd_obj['freq[%]'
#                ].loc[HCapp_train_dd_obj['freq[%]']
#                > cut_off_percentage].index.to_list()
#
#len(extreme_majority_class_acc), extreme_majority_class_acc

In [ ]:
#cut_off = 1000.
#
#too_many_unique_acc = LCacc_dd_obj['unique'
#                ].loc[LCacc_dd_obj['unique']
#                > cut_off].index.to_list()
#
#len(too_many_unique_acc), too_many_unique_acc

In [ ]:
#cut_off = 0.05 * size_acc[0]
#
#too_few_entries_acc = LCacc_dd_obj['count'
#                ].loc[LCacc_dd_obj['count']
#                < cut_off].index.to_list()
#
#len(too_few_entries_acc), too_few_entries_acc

In [ ]:
HCapp_dd_obj = HCapp_train_dd_obj.merge(HCapp_test_dd_obj, suffixes=[' train',' test'], left_index=True, right_index=True )
HCapp_dd_obj.drop(columns='top test') # identical results
HCapp_dd_obj.round(1)

,unique train,count train,top train,freq train,freq[%] train,unique test,count test,top test,freq test,freq[%] test
NAME_CONTRACT_TYPE,2,307511,Cash loans,278232,90.5,2,48744,Cash loans,48305,99.1
CODE_GENDER,3,307511,F,202448,65.8,2,48744,F,32678,67.0
FLAG_OWN_CAR,2,307511,N,202924,66.0,2,48744,N,32311,66.3
FLAG_OWN_REALTY,2,307511,Y,213312,69.4,2,48744,Y,33658,69.1
NAME_TYPE_SUITE,7,306219,Unaccompanied,248526,81.2,7,47833,Unaccompanied,39727,83.1
NAME_INCOME_TYPE,8,307511,Working,158774,51.6,7,48744,Working,24533,50.3
NAME_EDUCATION_TYPE,5,307511,Secondary / secondary special,218391,71.0,5,48744,Secondary / secondary special,33988,69.7
NAME_FAMILY_STATUS,6,307511,Married,196432,63.9,5,48744,Married,32283,66.2
NAME_HOUSING_TYPE,6,307511,House / apartment,272868,88.7,6,48744,House / apartment,43645,89.5
OCCUPATION_TYPE,18,211120,Laborers,55186,26.1,18,33139,Laborers,8655,26.1


### Other Checks and Modifications

In [ ]:
df = HCapp_train_dd

In [ ]:
nan_cat = ['EMERGENCYSTATE_MODE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'NAME_TYPE_SUITE', 'OCCUPATION_TYPE', 'WALLSMATERIAL_MODE']

binary_cat = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'] #, EMERGENCYSTATE_MODE


In [ ]:
YN_map = {'Y': 1, 'N': 0, 'F': 1, 'M': 0}
for feat in binary_cat:
    df[feat] = df[feat].map(YN_map)

In [ ]:
for feat in nan_cat:
    df[feat] = df[feat].mask((df[feat].isna()), 'NA')


In [ ]:
df_pd = df.compute()

In [ ]:
df_pd['NAME_CONTRACT_TYPE'].value_counts()

NAME_CONTRACT_TYPE
Cash loans         278232
Revolving loans     29279
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['CODE_GENDER'].value_counts()

CODE_GENDER
1.0    202448
0.0    105059
Name: count, dtype: int64

In [ ]:
df_pd['FLAG_OWN_CAR'].value_counts()

FLAG_OWN_CAR
0    202924
1    104587
Name: count, dtype: int64

In [ ]:
df_pd['FLAG_OWN_REALTY'].value_counts()

FLAG_OWN_REALTY
1    213312
0     94199
Name: count, dtype: int64

In [ ]:
df_pd['NAME_TYPE_SUITE'].value_counts()

NAME_TYPE_SUITE
Unaccompanied      248526
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
NA                   1292
Other_A               866
Group of people       271
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['NAME_INCOME_TYPE'].value_counts()

NAME_INCOME_TYPE
Working                 158774
Commercial associate     71617
Pensioner                55362
State servant            21703
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['NAME_EDUCATION_TYPE'].value_counts()

NAME_EDUCATION_TYPE
Secondary / secondary special    218391
Higher education                  74863
Incomplete higher                 10277
Lower secondary                    3816
Academic degree                     164
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['NAME_FAMILY_STATUS'].value_counts()

NAME_FAMILY_STATUS
Married                 196432
Single / not married     45444
Civil marriage           29775
Separated                19770
Widow                    16088
Unknown                      2
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['NAME_HOUSING_TYPE'].value_counts()

NAME_HOUSING_TYPE
House / apartment      272868
With parents            14840
Municipal apartment     11183
Rented apartment         4881
Office apartment         2617
Co-op apartment          1122
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['OCCUPATION_TYPE'].value_counts()

OCCUPATION_TYPE
NA                       96391
Laborers                 55186
Sales staff              32102
Core staff               27570
Managers                 21371
Drivers                  18603
High skill tech staff    11380
Accountants               9813
Medicine staff            8537
Security staff            6721
Cooking staff             5946
Cleaning staff            4653
Private service staff     2652
Low-skill Laborers        2093
Waiters/barmen staff      1348
Secretaries               1305
Realty agents              751
HR staff                   563
IT staff                   526
Name: count, dtype: int64[pyarrow]

In [ ]:
#df['ORGANIZATION_TYPE'].value_counts().compute()

In [ ]:
df_orga = df['ORGANIZATION_TYPE']
df_orga = df_orga.mask((df_orga.str.startswith('Business')), 'Business')
df_orga = df_orga.mask((df_orga.str.startswith('Trade')), 'Trade')
df_orga = df_orga.mask((df_orga.str.startswith('Industry')), 'Industry')
df_orga = df_orga.mask((df_orga.str.startswith('Transport')), 'Transport')
df_orga = df_orga.mask((df_orga.isin(['Emergency'])), 'Medicine')
df_orga = df_orga.mask((df_orga.isin(['Security Ministries',
                                      'Police', 'Military'])), 'Government')
df_orga = df_orga.mask((df_orga.isin(['University', 'School',
                                      'Kindergarten', 'Culture'])), 'Education')
df_orga = df_orga.mask((df_orga.isin(['Construction', 'Realtor'])), 'Housing')
df_orga = df_orga.mask((df_orga.isin(['Restaurant', 'Hotel'])), 'Hotel_Restaurant')
df_orga = df_orga.mask((df_orga.isin(['Services', 'Cleaning'])), 'Services_Cleaning')
df_orga = df_orga.mask((df_orga.isin(['Insurance', 'Legal Services',
                                      'Security'])),
                       'Insurance_Legal_Services_Security')
df_orga = df_orga.mask((df_orga.isin(['Telecom'])), 'Mobile')
df_orga = df_orga.mask((df_orga.isin(['Advertising', 'Religion'])), 'Other')

df = df.drop(columns='ORGANIZATION_TYPE')
df['ORGANIZATION_TYPE_red'] = df_orga
df['ORGANIZATION_TYPE_red'].value_counts().compute()

ORGANIZATION_TYPE_red
Business                             84529
XNA                                  55374
Self-employed                        38412
Education                            17479
Government                           17353
Other                                17197
Trade                                14315
Industry                             14311
Medicine                             11753
Housing                              10075
Transport                             8990
Insurance_Legal_Services_Security     4149
Hotel_Restaurant                      2777
Bank                                  2507
Agriculture                           2454
Postal                                2157
Services_Cleaning                     1835
Electricity                            950
Mobile                                 894
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['FONDKAPREMONT_MODE'].value_counts()

FONDKAPREMONT_MODE
NA                       210295
reg oper account          73830
reg oper spec account     12080
not specified              5687
org spec account           5619
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['HOUSETYPE_MODE'].value_counts()

HOUSETYPE_MODE
NA                  154297
block of flats      150503
specific housing      1499
terraced house        1212
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['WALLSMATERIAL_MODE'].value_counts()

WALLSMATERIAL_MODE
NA              156341
Panel            66040
Stone, brick     64815
Block             9253
Wooden            5362
Mixed             2296
Monolithic        1779
Others            1625
Name: count, dtype: int64[pyarrow]

In [ ]:
df_pd['EMERGENCYSTATE_MODE'].value_counts()

EMERGENCYSTATE_MODE
No     159428
NA     145755
Yes      2328
Name: count, dtype: int64[pyarrow]

### EXT_SOURCE

In [ ]:
df['EXT_SUM'] = df['EXT_SOURCE_1'] + df['EXT_SOURCE_2'] + df['EXT_SOURCE_3']

df['EXT_SUM2'] = df['EXT_SOURCE_1']**2 + df['EXT_SOURCE_2']**2 + df['EXT_SOURCE_3']**2

df['EXT_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

df['EXT_RATIO'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] / df['EXT_SOURCE_3']


### AMT ratios

In [ ]:
df['AMT_ANNUITY_CREDIT_ratio'] = df['AMT_ANNUITY']/(df['AMT_CREDIT'])# + 0.0001)

df['AMT_GOODS_PRICE_CREDIT_ratio'] = df['AMT_GOODS_PRICE']/(df['AMT_CREDIT'])

df = df.replace([np.inf, -np.inf], np.nan)

In [ ]:
df.loc[(df.AMT_ANNUITY>0)&
        (df.AMT_CREDIT>0)
        ][['SK_ID_CURR', 'AMT_ANNUITY',
           'AMT_ANNUITY_CREDIT_ratio',
           'AMT_GOODS_PRICE_CREDIT_ratio'
          ]].compute().head(50).round(3)

,SK_ID_CURR,AMT_ANNUITY,AMT_ANNUITY_CREDIT_ratio,AMT_GOODS_PRICE_CREDIT_ratio
0,100002,24700.5,0.061,0.863
1,100003,35698.5,0.028,0.873
2,100004,6750.0,0.050,1.000
3,100006,29686.5,0.095,0.950
4,100007,21865.5,0.043,1.000
5,100008,27517.5,0.056,0.927
6,100009,41301.0,0.026,0.894
7,100010,42075.0,0.028,1.000
8,100011,33826.5,0.033,0.896
9,100012,20250.0,0.050,1.000


### debt-to-income ratio

In [ ]:
df['dti'] = df['AMT_ANNUITY']/ df['AMT_INCOME_TOTAL']


### Bundle AMT_REQ_CREDIT_BUREAU

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_TOTAL'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'] + \
                                    df['AMT_REQ_CREDIT_BUREAU_DAY'] + \
                                    df['AMT_REQ_CREDIT_BUREAU_WEEK'] + \
                                    df['AMT_REQ_CREDIT_BUREAU_MON'] + \
                                    df['AMT_REQ_CREDIT_BUREAU_QRT'] + \
                                    df['AMT_REQ_CREDIT_BUREAU_YEAR']

### Sum Document Flags

In [ ]:
docs = ['FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']


df['FLAG_DOCUMENT_SUM'] = df[docs].sum(axis=1)


In [ ]:
#num_feat = df.select_dtypes(include=np.number).columns.tolist()
#num_feat

### Building means

In [ ]:
building_avg = ['APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG']

df['building_avg'] = df[building_avg].mean(axis=1)

building_mode = ['APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE']

df['building_mode'] = df[building_mode].mean(axis=1)


building_medi = ['APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI']

df['building_medi'] = df[building_medi].mean(axis=1)


In [ ]:
del df_pd

In [ ]:
df_final = df

# Export

In [ ]:
%%capture
! mkdir home_folder+'cleaned/'
df_final.to_csv(home_folder+'cleaned/HC_app_train_cleaned.csv',
                 index=False, single_file = True)

## Merge with Sub-Tables

In [ ]:
HCapp_target = dd.read_csv(home_folder+'cleaned/HC_app_train_cleaned.csv')
HCapp_prev = dd.read_csv(home_folder+'cleaned/HC_previous_app_cleaned.csv')
HCapp_bureau = dd.read_csv(home_folder+'cleaned/HC_bureau_cleaned.csv')

HCapp_bureau['flag_table_bureau'] = 1

In [ ]:
app_features = HCapp_target.columns
prev_features = HCapp_prev.columns
bureau_features = HCapp_bureau.columns


In [ ]:
df_target_prev = HCapp_target.merge(HCapp_prev, on = ['SK_ID_CURR'],
                                    suffixes = ('', '_prev'),
                                 how = 'left')

In [ ]:
df_target_prev_bureau = df_target_prev.merge(HCapp_bureau, on = ['SK_ID_CURR'],
                                             suffixes = ('', '_bureau'),
                                 how = 'left')

In [ ]:
size_df_final = [df_target_prev_bureau.shape[0].compute(),  df_target_prev_bureau.shape[1]]

print('The combined dataset has', size_df_final[0], 'rows and', size_df_final[1], 'features.')


The combined dataset has 307511 rows and 289 features.


In [ ]:
len(df_target_prev_bureau), len(df_target_prev), len(HCapp_target)

(307511, 307511, 307511)

In [ ]:
nan_overview_df = dfunc.nan_type_overview_dd(df_target_prev_bureau, size_df_final[0])
nan_overview_df.round(1).style.background_gradient(cmap="Blues")

,type,NaN[abs],NaN[%]
SK_ID_CURR,int64,0,0.000000
TARGET,int64,0,0.000000
NAME_CONTRACT_TYPE,string,0,0.000000
CODE_GENDER,float64,4,0.000000
FLAG_OWN_CAR,int64,0,0.000000
FLAG_OWN_REALTY,int64,0,0.000000
CNT_CHILDREN,int64,0,0.000000
AMT_INCOME_TOTAL,float64,0,0.000000
AMT_CREDIT,float64,0,0.000000
AMT_ANNUITY,float64,12,0.000000


## debt-income-ratio all

In [ ]:
df_target_prev_bureau['dti_all'] = (df_target_prev_bureau['AMT_ANNUITY_sum_active_bureau'] + \
             df_target_prev_bureau['AMT_ANNUITY_sum_active_prev'] + \
             df_target_prev_bureau['AMT_ANNUITY'])/ df_target_prev_bureau['AMT_INCOME_TOTAL']

df_target_prev_bureau['dti_prev'] = (df_target_prev_bureau['AMT_ANNUITY_sum_active_prev'] + \
             df_target_prev_bureau['AMT_ANNUITY'])/ df_target_prev_bureau['AMT_INCOME_TOTAL']

df_target_prev_bureau.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,ORGANIZATION_TYPE_red,EXT_SUM,EXT_SUM2,EXT_PROD,EXT_RATIO,AMT_ANNUITY_CREDIT_ratio,AMT_GOODS_PRICE_CREDIT_ratio,dti,AMT_REQ_CREDIT_BUREAU_TOTAL,FLAG_DOCUMENT_SUM,building_avg,building_mode,building_medi,AMT_ANNUITY_prev,AMT_APPLICATION,AMT_CREDIT_prev,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE_prev,HOUR_APPR_PROCESS_START_prev,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,WEEKDAY_APPR_PROCESS_START_ordinal,AMT_ANNUITY_plain,AMT_ANNUITY_diff,int_rate_plain,int_rate_plain_diff,AMT_ANNUITY_CREDIT_ratio_prev,AMT_ANNUITY_plain_ratio,AMT_GOODS_PRICE_CREDIT_ratio_prev,AMT_DOWN_PAYMENT_CREDIT_ratio,CNT_PAYMENT_inv,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,NAME_CONTRACT_TYPE_Cash_loans,NAME_CONTRACT_TYPE_Consumer_loans,NAME_CONTRACT_TYPE_Revolving_loans,NAME_CONTRACT_TYPE_XNA,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,NAME_PAYMENT_TYPE_Cash_through_the_bank,NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,NAME_PAYMENT_TYPE_Non-cash_from_your_account,NAME_PAYMENT_TYPE_XNA,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group_of_people,NAME_TYPE_SUITE_NA,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse,_partner",NAME_TYPE_SUITE_Unaccompanied,NAME_CLIENT_TYPE_New,NAME_CLIENT_TYPE_Refreshed,NAME_CLIENT_TYPE_Repeater,NAME_CLIENT_TYPE_XNA,NAME_PORTFOLIO_Car

## Export Merged

In [ ]:
%%capture
! mkdir home_folder+'cleaned/'
df_target_prev_bureau.to_csv(home_folder+'cleaned/HC_app_train_combined.csv',
                 index=False, single_file = True)